In [1]:
!pip install -U spacy --q
!python -m spacy download ru_core_news_lg --q
!pip install pymorphy3 --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 66.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.7.1 requires spacy<3.8.0,>=3.7.2, but you have spacy 3.8.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 47.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependenci

In [2]:
import spacy
from spacy.tokens import Span

from pymorphy3 import MorphAnalyzer

import types
from types import NoneType

In [54]:
text = 'Иван Иванов перестал работать по рекомендации Всемирной Организации Здравоохранения. По этой причине его уволили из Яндекса.'

In [55]:
nlp = spacy.load("ru_core_news_lg")
doc = nlp(text)

morph = MorphAnalyzer()

In [48]:
def head_in_named_entity(doc, span): # на вход подаются документ и именованная сущность, которая в нём содержится
    span_parts = span.text.split(' ')
    head = None
    heads = [[], []] # [[token], [head]]
    for span_part in span_parts:
        for token in doc: # перебираем токены, потому что именно они, в отличие от строк, содержат всю информацию
            if span_part == token.text:
                heads[0].append(token)
                heads[1].append(token.head)
    for i in range(len(span_parts)):
        if heads[1][i] not in heads[0]: # вершиной является то, что не зависит от других слов, входящих в именованную сущность
            head = heads[0][i]
    return head, [_.split('=')[1] for _ in str(head.morph).split('|')], head.head, head.dep_, heads

In [49]:
def normalize_noun_phrase(doc, np): # на вход подаётся документ и именная группа
    head, morphology, parent, dep, np_parts = head_in_named_entity(doc, np)
    ana = morph.parse(head.text)[0]
    res = ''
    for i, np_part in enumerate(np_parts[0]):
        np_part_head = np_parts[1][i]
        if np_part == head:
            np_part = ana.normal_form
        else:
            np_part = morph.parse(np_part.text)[0]
            pos = str(np_part.tag).split(',')[0].split(' ')[0]
            if pos == 'ADJF' and np_part_head == head:
                gender, number = str(ana.normalized.tag).split(',')[2].split()
                np_part = np_part.inflect({gender, 'nomn'})[0]
            else:
                np_part = np_part.word
        res += np_part + ' '
    return res.strip()

Функция ниже заменяет местоимения на именные группы и выделяет именные группы вместе с глаголами, от которых они зависят.

In [58]:
def get_syntactic_relations(doc):
    chunks = [] # тут содержатся как именованные сущности, так и просто существительные
    res = {} # {(индекс первого символа, индекс последнего символа): чанк в тексте, нормализованный чанк, родитель чанка, тип зависимости}
    for ent in doc.ents: # добавляем именованные сущности
        chars = (ent.start_char, ent.end_char)
        chunks.append((chars, ent, ) + head_in_named_entity(doc, ent)[1:-1])
        res[chars] = (ent.text, normalize_noun_phrase(doc, ent), chunks[-1][3], chunks[-1][4])
    for token in doc: # добавляем существительные
        if token.pos_ == 'NOUN' or token.pos_ == 'PROPN':
            morph = [_.split('=')[1] for _ in str(token.morph).split('|')]
            chars = (token.idx, token.idx + len(token.text))
            chunks.append((chars, token, morph, token.head, token.dep_))
            res[chars] = (token.text, str(token.lemma_), chunks[-1][3], chunks[-1][4])
    chunks.sort(key=lambda x: x[0])
    for token in doc: # решаем анафору
        if token.pos_ == 'PRON':
            morph = [_.split('=')[1] for _ in str(token.morph).split('|')]
            for chunk in chunks:
                if chunk[0][0] < token.idx and chunk[2][2:4] == morph[1:3]:
                    chars = (token.idx, token.idx + len(token.text))
                    res[chars] = (token.text, normalize_noun_phrase(doc, chunk[1]), chunk[3], chunk[4])
    print(res)

In [57]:
get_syntactic_relations(doc)

{(0, 11): ('Иван Иванов', 'иван иванов', перестал, 'nsubj'), (46, 83): ('Всемирной Организации Здравоохранения', 'всемирная организация здравоохранения', рекомендации, 'nmod'), (116, 123): ('Яндекса', 'яндекс', уволили, 'obl'), (0, 4): ('Иван', 'иван', перестал, 'nsubj'), (5, 11): ('Иванов', 'иванов', Иван, 'appos'), (33, 45): ('рекомендации', 'рекомендация', работать, 'obl'), (56, 67): ('Организации', 'организация', рекомендации, 'nmod'), (68, 83): ('Здравоохранения', 'здравоохранение', Организации, 'nmod'), (93, 100): ('причине', 'причина', уволили, 'obl'), (101, 104): ('его', 'иванов', Иван, 'appos')}


Можно в будущем не включать слова, входящие в чанк, если нам не интересны связи между частями чанка.